In [6]:
import os
import glob
import tkinter as tk
from tkinter import filedialog
import struct
import pandas as pd
import re
import pickle
import numpy as np

def select_folder():
    root = tk.Tk()
    root.withdraw()  # GUIを隠す
    folder_path = filedialog.askdirectory()
    return folder_path

def find_d0_files(folder_path):
    d0_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if re.search(r'\.[dD]\d+\.?\w*', file):  # .d0または.D0を含む拡張子を探す（.d00.tmpのような形式にも対応）
                d0_files.append(os.path.join(root, file))
    return d0_files

def load_sensor_data(file_path):
    with open(file_path, 'rb') as file:
        data = file.read()

    sensor_data = [data[i:i+8*2] for i in range(408, len(data), 8*2)]
    sensor_values = [[] for _ in range(8)]
    for sensor in sensor_data:
        for i in range(8):
            sensor_value = unsigned2signed(int.from_bytes(sensor[i*2:i*2+2], 'little'))
            sensor_values[i].append(sensor_value)
    
    return sensor_values

def unsigned2signed(unsigned_value):
    if unsigned_value >= 32768:
        return unsigned_value - 65536
    else:
        return unsigned_value

def create_data_pickle(data_dir, folder_name, sensor_values, filename):  # sensor_values と filename を引数に追加
    """
    指定されたフォルダ内の波形データ、ラベル、ラベル位置をpickleファイルに保存します。

    Args:
        data_dir (str): データディレクトリのパス
        folder_name (str): 処理対象のフォルダ名
        sensor_values (list): センサー値のリスト
        filename (str): ファイル名
    """

    judge_dir = r"C:\Users\r-fujita\Desktop\藤澤さんデータ3課\JUDGE_Excel"  # JUDGEフォルダのパス
    judge_filepath = os.path.join(judge_dir, f"{folder_name}.xlsx")  # JUDGEファイルのパス

    # 解析データを読み込み
    df_judge = pd.read_excel(judge_filepath, header=None)

    data = []
    try:
        # センサー値を NumPy 配列に変換
        waveform_data = np.array(sensor_values).T  # 転置して波形データの形式に合わせる

        # ファイル名からA列とB列の値を取得
        file_parts = filename.split('-')
        a_col = int(file_parts[0])
        b_col = int(file_parts[1])

        # ラベル位置を取得 (複数ラベルに対応)
        label_positions = df_judge[
            (df_judge[0] == a_col) & (df_judge[1] == b_col)
        ][11].values

        # ラベルを生成
        labels = np.where(label_positions == 0, 0, 1)

        # データをリストに追加
        data.append({
            'folder_name': folder_name,
            'filename': filename + '.xlsx',  # 拡張子を追加
            'waveform_data': waveform_data,
            'labels': labels,
            'label_positions': label_positions
        })

    except Exception as e:
        print(f"Error processing {filename}: {e}")

    # pickleファイルに保存
    save_dir = os.path.join(data_dir, 'pickle_data')
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{folder_name}.pickle")
    with open(save_path, 'wb') as f:
        pickle.dump(data, f)

def main():
    base_folder = select_folder()
    if not base_folder:
        print("フォルダが選択されませんでした。")
        return

    data_dir = r"C:\Users\r-fujita\Desktop\藤澤さんデータ3課"  # データディレクトリのパス
    save_base_folder = os.path.join(data_dir, 'pickle_data')  # 保存先フォルダ
    os.makedirs(save_base_folder, exist_ok=True)

    for folder_name in os.listdir(base_folder):
        folder_path = os.path.join(base_folder, folder_name)
        if os.path.isdir(folder_path):
            d0_files = find_d0_files(folder_path)
            if not d0_files:
                print(f"{folder_path} 内に拡張子に.d0を含むファイルが見つかりませんでした。")
                continue

            data = []
            for file_path in d0_files:
                file_name = os.path.basename(file_path).split('.')[0]  # 拡張子を除いたファイル名
                sensor_values = load_sensor_data(file_path)
                
                # create_data_pickle 関数を呼び出し、センサー値とファイル名を渡す
                create_data_pickle(data_dir, folder_name, sensor_values, file_name)

if __name__ == "__main__":
    main()

KeyboardInterrupt: 